In [1]:
from IPython.display import display, Markdown, clear_output
import ipywidgets as ipw
import json
import subprocess
import atexit
import os
import shutil
from ai_agent import ai_agent
from src import widgets, utils

In [2]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])
GOOGLE_API_KEY = utils.read_json("/home/jovyan/gemini_api.json")

In [2]:
from ai_agent.tools import inventory_tools

x = inventory_tools.SubstanceObject(
    empa_number = 650,
    batch = "a",
    molecules = [
        inventory_tools.MoleculeObject(
            smiles = "CCO"
        )
    ]
)
x

inventory_tools.get_substances_by_attributes(x)

[{'permId': '20250717102536145-2290',
  'type': 'SUBSTANCE',
  'properties': {'name': 'gr1mv467',
   'molecules': [{'permId': '20250717093541016-370',
     'type': 'MOLECULE',
     'properties': {'name': '650',
      'smiles': 'N#Cc1c2ccccc2c(-c2c3ccccc3c(C#N)c3ccc(Br)cc23)c2cc(Br)ccc12',
      'sum_formula': 'C30H14Br2N2',
      'cas_number': None,
      'iupac_name': None,
      'comments': None},
     'registration_date': '2025-07-17 11:35:41'}],
   'empa_number': '650',
   'batch': 'a',
   'vial': None,
   'evaporation_temperatures': None,
   'purity': None,
   'substance_type': None,
   'amount': None,
   'chemist_own_name': None,
   'comments': 'Brought by Paula from Dimas group',
   'location': None,
   'special_storage_conditions': None,
   'package_opening_date': None,
   'object_status': None,
   'supplier': None,
   'synthesised_by': None,
   'supplier_own_name': None,
   'receive_date': '2025-01-14'},
  'registration_date': '2025-07-17 12:25:36'},
 {'permId': '2025071710380

In [3]:
openbis_agent = ai_agent.OpenBISAgent(
    google_api_key=GOOGLE_API_KEY["api_key"]
)

In [ ]:
# x = openbis_agent.ask_question("Which experiments are available? Give me a summary of them")
# x = openbis_agent.ask_question("Is there an experiment that uses molecule 704a? If yes, give me a summary of it including the name of sample that was used and the sample that was produced by each process step and a small description of the actions and observables.")
# x = openbis_agent.ask_question("Is there substance 704a?")
# x = openbis_agent.ask_question("Is there a simulation in openBIS? If yes give me a summary")
# x = openbis_agent.ask_question("Considering the geometry optimisation, what was the used molecule?")
# x = openbis_agent.ask_question("Find me all the 2d materials in openBIS")
# x = openbis_agent.ask_question("Find me objects created between 17th July 2025 and 18th July 2025")
# x = openbis_agent.ask_question("Is instrument CreaTec THz-STM in openBIS? If yes, how many pumps does it have?")
# x = openbis_agent.ask_question("What object types exist in openBIS?")
# x = openbis_agent.ask_question("Yes. No filters give me all that you find")

In [9]:
x = openbis_agent.ask_question("Is there a substance with smiles CCO?")

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 44
}
].


ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 41
}
]

In [7]:
x

[HumanMessage(content='Is there a substance with empa number 650 and batch a?', additional_kwargs={}, response_metadata={}, id='db613d5b-65fe-456c-8dfc-af6b81f96056'),
 AIMessage(content="Yes, there is a substance with Empa number 650 and batch 'a'. Its name is gr1mv467 and its permId is 20250717102536145-2290.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--0b517b13-1fbc-4dcb-ba52-f8ef47680331-0', usage_metadata={'input_tokens': 402026, 'output_tokens': 57, 'total_tokens': 402297, 'input_token_details': {'cache_read': 400970}})]

In [8]:
print("Question:", x[0].content)
print("Answer:", x[-1].content)

Question: Is there a substance with empa number 650 and batch a?
Answer: Yes, there is a substance with Empa number 650 and batch 'a'. Its name is gr1mv467 and its permId is 20250717102536145-2290.


In [10]:
openbis_agent.get_graph_replay()

Num Messages:  5 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  4 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  3 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  2 Next:  ('chatbot',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('__start__',)
--------------------------------------------------------------------------------
Num Messages:  1 Next:  ('chatbot',)
-----------

In [ ]:
exp = OPENBIS_SESSION.get_experiment("20250717115224276-3595")
exp.get_objects()[0].registrationDate

In [ ]:
all_messages = []
for state in openbis_agent.graph.get_state_history(openbis_agent.agent_config):
    all_messages.extend(state.values["messages"])
all_messages

In [ ]:
obj_type = OPENBIS_SESSION.get_object_types()[10]
properties_df = obj_type.get_property_assignments().df
properties_codes = properties_df["code"]
properties_datatypes = properties_df["dataType"]
properties_info = []
for code, datatype in zip(properties_codes, properties_datatypes):
    property_info = {
        "code": code.lower(),
        "datatype": datatype.lower()
    }
    properties_info.append(property_info)

properties_info